In [13]:
import os
import pandas as pd
import numpy as np
from sklearn.utils import shuffle
import torch.nn as nn
import torch
import torch.optim as optim
import matplotlib.pyplot as plt
import torch.nn.functional as F

In [14]:
np.random.seed(0)
data = pd.read_csv("..\\Data\\cle\\Rand_train.csv", sep=',')
data = shuffle(data).reset_index(drop=True)
data_val = pd.read_csv("..\\Data\\cle\\Rand_valid.csv", sep=',')

In [15]:
X = data.iloc[:,1:]
Y = data.iloc[:,0]
X_val = data_val.iloc[:,1:]
Y_val = data_val.iloc[:,0]

X2 = X
X2_val = X_val

In [16]:
cut0 = 0
cut1 = 1146
y10 = (Y == cut0) * 1
y11 = ((Y > cut0) & (Y <= cut1)) * 1
y12 = (Y > cut1) * 1
y21 = y11 * Y
y22 = y12 * Y
Y2 = np.stack((y10, y11, y12, y21, y22), axis=-1)


y10 = (Y_val == cut0) * 1
y11 = ((Y_val > cut0) & (Y_val <= cut1)) * 1
y12 = (Y_val > cut1) * 1
y21 = y11 * Y_val
y22 = y12 * Y_val
Y2_val = np.stack((y10, y11, y12, y21, y22), axis=-1)

In [18]:
n_batches = 6
batch_size = X2.shape[0] // n_batches

In [19]:
idx = list(X2.index)
idx = idx[:n_batches*batch_size]

In [25]:
bi = np.random.choice(idx, batch_size, replace=False)
X_batch = X2.loc[bi]
Y_batch = Y2[bi]       
X_batch.shape

(1073, 16)

In [26]:
Y_batch.shape

(1073, 5)

In [ ]:
from fit_nn import fit, comb_error , mae_error

In [ ]:
class Net1(nn.Module):
    def __init__(self):
        super(Net1, self).__init__()
        
        self.fc11 = nn.Linear(16, 24, bias = True)
        self.fc12 = nn.Linear(24, 3, bias = True)
       
        
        self.fc21 = nn.Linear(16, 24, bias=True)
        self.fc22 = nn.Linear(24, 5, bias=True)
        self.fc23 = nn.Linear(5, 2, bias=True)
                
    def forward(self, x):
        x1 = torch.tanh(self.fc11(x))
        x1 = torch.sigmoid(self.fc12(x1))
        
        x2 = self.fc21(x)
        x2 = torch.tanh(self.fc22(x2))
        x2 = torch.exp(self.fc23(x2))       
        
        x3 = torch.cat((x1, x2), 1)
        return x3

In [ ]:
n_epochs = 2000
lr = 0.0001
verbose = 1
n_batches = 6
batch_to_avg = 2
ep_to_check = 5
clipping = 0.20
sig2 = 2 * 950000
PATH = 'Data\\checkpoints\\mytraining10.pt'
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [ ]:
def m_error(output, target):
    return comb_error(output, target, sig2)

In [ ]:
def weights_init_normal(m):     
    classname = m.__class__.__name__
    torch.manual_seed(0)
    if classname.find('Linear') != -1:
        n = m.in_features
        y = 1.0/np.sqrt(n)
        m.weight.data.normal_(0, y)
        if m.bias is not None:
            m.bias.data.normal_(0, y)

In [ ]:
def weights_init_uniform_rule(m):
    classname = m.__class__.__name__
    torch.manual_seed(0)
    if classname.find('Linear') != -1:
        n = m.in_features
        y = 1.0/np.sqrt(n)
        m.weight.data.uniform_(-y, y)
        m.bias.data.data.uniform_(-y, y)

In [ ]:
net = Net1()
#optimizer = optim.SGD(net.parameters(), lr=lr, momentum=0.9)  
optimizer = optim.Adam(net.parameters(), lr=lr)

net.apply(weights_init_normal)
#net.apply(weights_init_uniform_rule)

#PATH = 'Data\\checkpoints\\mytraining10.pt'
#net.load_state_dict(torch.load(PATH, map_location = device))
    
a, b, c = fit(X=X2, X_val=X2_val, Y=Y2, Y_val=Y2_val, net=net, optimizer=optimizer, error=m_error, val_error = mae_error,
                n_epochs=n_epochs, n_batches=n_batches, batch_to_avg=batch_to_avg, 
                    ep_to_check=ep_to_check, clipping=clipping, PATH=PATH, device=device, 
                                            verbose=verbose, min_val_loss = float('inf'))

#(PENDIENTE)